<a href="https://colab.research.google.com/github/rpezoa/Intro_XAI/blob/main/07_CAM_LoadModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mapa de Activación por Clase (CAM)

Los Class Activation Maps son una técnica de visualización que nos permite entender qué regiones de una imagen fueron más importantes para que una red neuronal convolucional (CNN) realice una clasificación específica.

**Funcionamiento básico:**
1. Utiliza las activaciones de la última capa convolucional
2. Aplica Average Pooling para obtener pesos por canal
3. Combina linealmente estos mapas usando los pesos de la capa fully-connected
4. Genera un mapa de calor que resalta las regiones relevantes

## Importación de librerías

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import Input
from tensorflow.keras.utils import to_categorical

## Entrenamiento
**En esta sección se entrena una arquitectura basada en VGG16. Se incluye el código correspondiente, aunque, para agilizar la ejecución del proyecto, más adelante se utilizan los pesos ya entrenados de este modelo con el fin de ahorrar tiempo. Llega hasta la sección de Carga de Modelo**
## Arquitectura VGG16 Modificada

La red VGG16 original se modifica para permitir la generación de CAM:
1. **Eliminamos las capas fully-connected finales**
2. **Añadimos Global Average Pooling:**  
   Reduce cada mapa de características a un solo valor promediando espacialmente
3. **Capa fully-connected final:**  
   Con 10 unidades (una por clase en CIFAR-10) sin función de activación

**¿Por qué Global Average Pooling?**
- Conserva información espacial de las últimas activaciones
- Permite combinar los mapas de características linealmente

In [14]:
#  Cargar y preparar datos
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# Convertir etiquetas a one-hot
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Función de preprocesamiento
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normaliza a [0,1]
    image = tf.image.resize(image, [224, 224])  # Redimensiona a 224x224 para VGG16
    return image, label

# Convertir a datasets de TensorFlow y aplicar preprocesamiento
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train_cat)).map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test_cat)).map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Lista de clases (por si haces predicciones)
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Crear modelo VGG16 + GAP + Dense
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
for layer in base_model.layers:
    layer.trainable = False  # congelar base

x = GlobalAveragePooling2D()(base_model.output)
output = Dense(10)(x)  # logits para CAM
model = Model(inputs=base_model.input, outputs=output)

## Carga del modelo entrenado

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('modelo_vgg16_cifar10.h5')

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:

!gdown https://drive.google.com/uc?id=1zgz_xxzV_fMIGG3-1aocXnIMEPLgD8FE

In [7]:
from tensorflow.keras.models import load_model

model = load_model('modelo_vgg16_cifar10.h5')

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

## Mapa de Activación por Clase (Class Activation Map - CAM)

Una vez entrenado el modelo, es posible visualizar qué regiones de la imagen influyeron más en la decisión del modelo usando CAM.

Para ello:
- Se extrae la activación de la última capa convolucional (`block5_conv3`) y la salida final del modelo (logits).
- Se identifica la clase predicha.
- Luego se toma el vector de pesos de la capa `Dense` (GAP → salida) correspondiente a la clase predicha y se realiza un producto punto con los mapas de activación.
- Esto genera un mapa de calor espacial que resalta las áreas más relevantes de la imagen para esa predicción.

El resultado se normaliza y se redimensiona al tamaño original de la imagen para su visualización posterior.


### Imagen test a evaluar

In [ ]:
# Seleccionar una imagen de prueba
img_idx = 0
test_img = x_test[img_idx:img_idx+1]
test_img = tf.image.resize(test_img, [224, 224])
test_img = test_img / 255.0  # Normalizar

# Obtener etiqueta verdadera
true_label = class_names[int(y_test[img_idx].item())]

# Mostrar imagen
plt.imshow(test_img[0])
plt.title(f"Etiqueta verdadera: {true_label}")
plt.axis('off')
plt.show()

In [ ]:
# Mostrar imagen
plt.figure(figsize=(1,1))
plt.imshow(x_test[0])
plt.title(f"Etiqueta verdadera: {true_label}")
plt.axis('off')
plt.show()

In [9]:
# Obtener activaciones y logits
conv_layer = model.get_layer('block5_conv3')
conv_model = Model(inputs=model.input, outputs=[conv_layer.output, model.output])
conv_output, logits = conv_model.predict(test_img)
pred_class = np.argmax(logits[0])
pred_label = class_names[pred_class]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 745ms/step


In [10]:
# Calcular CAM
gap_weights = model.layers[-1].get_weights()[0]  # shape (512, 10)
cam = np.dot(conv_output[0], gap_weights[:, pred_class])  # (14,14)
cam = np.maximum(cam, 0)
cam = cam / cam.max()
cam = cv2.resize(cam, (224, 224))

In [11]:
# Superponer mapa sobre imagen original
img_orig = (test_img[0].numpy() * 255).astype(np.uint8)
img_orig = cv2.cvtColor(img_orig, cv2.COLOR_RGB2BGR)
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
superimposed = cv2.addWeighted(img_orig, 0.6, heatmap, 0.4, 0)

In [ ]:
# Mostrar resultado
plt.figure(figsize=(6, 6))
plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
plt.title(f"CAM para clase predicha: {pred_label}\n(Clase real: {true_label})")
plt.axis('off')
plt.show()